# Double pendulum using Lagrange's equation

Defines a doublePendulum class that is used to generate basic pendulum plots from solving Lagrange's equations.

* Last revised 25-Apr-2019 by Abasi Brown (brown.7146@buckeyemail.osu.edu). *

## Euler-Lagrange equation

For a double pendulum, the Lagrangian with generalized coordinate $\phi_{1}$ & $\phi_{2}$ is

$\begin{align}
  \mathcal{L} = \frac12 (m_{1} + m_{2})\ L_{1}^2 \dot\phi_{1}^2 
  + \frac12 m_{2} L_{2}^2 \dot\phi_{2}^2 + m_{2}gL_{1}L_{2}\dot\phi_{1}
  \dot\phi_{2}\cos(\phi_{1} - \phi_{2})
\end{align}$

The Euler-Lagrange equations are:

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_{1}} =  (m_{1} + m_{2}) L_{1}
 \ddot \phi_{1} + m_{2}L_{2}\ddot \phi_{2}cos(\phi_{1} - \phi_{2})- m_{2}L_{2}
 \dot\phi_{2}sin(\phi_{1} - \phi_{2})(\dot\phi_{1} - \dot\phi_{2})
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial\phi_{1}}
 = -g(m_{1} + m_{2})sin(\phi_{1})-m_{2}L_{2}\dot\phi_{1}\dot\phi_{2}sin(\phi_{1} - \phi_{2})
 \end{align}$
 
 $\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_{1}} =  m_{2}L_{2}\ddot \phi_{2} +
 m_{2}L_{1}\ddot \phi_{1}cos(\phi_{1} - \phi_{2})- m_{2}L_{1}\dot\phi_{1}sin(\phi_{1} - \phi_{2})
 (\dot\phi_{1} - \dot\phi_{2})
  \;
\end{align}$

$\begin{align}
 \frac{\partial\mathcal L}{\partial\phi_{2}}
 = m_{2}L_{1}\dot\phi_{1}\dot\phi_{2}sin(\phi_{1} - \phi_{2}) - m_{2}gsin\phi_{2}
 \end{align}$
 
 From these equations we obtain the following values for $\ddot\phi_{1}$ & $\ddot\phi_{2}$:
 $\begin{align}
 \ddot\phi_{1} & =  \bigl(gm_{1}sin(\phi_{1}) + gm_{2}sin(\phi_{1}) + L_{2}m_{2}\dot\phi_{2}^2 sin(\phi_{1} - \phi_{2}) \\
   & \ \ + L_{1}L_{2}m_{2}\dot\phi_{1}^2 cos(\phi_{1} - \phi_{2}) sin(\phi_{1} - \phi_{2})-gm_{2}cos(\phi_{1} - \phi_{2}) sin(\phi_{2})\bigr)  / \bigl(L_{1}(-m_{1}-m_{2}sin^2(\phi_{1} - \phi_{2}))\bigr)
 \end{align}$
 
  $\begin{align}
 \ddot\phi_{2} & =\bigl(gm_{1}sin(\phi_{2}) + gm_{2}sin(\phi_{2}) - gm_{1}cos(\phi_{1} - \phi_{2})sin(\phi_{1})
 -gm_{2}cos(\phi_{1} - \phi_{2})sin(\phi_{1}) \\
  & \ \ -L_{1}L_{2}\dot\phi_{1}^2sin(\phi_{1} - \phi_{2})(m_{1}+m_{2})-
 L_{2}m_{2}\dot\phi^2cos(\phi_{1} - \phi_{2})sin(\phi_{1} - \phi_{2})\bigr) /\bigl(L_{2}(-m_{1}-m_{2}sin^2(\phi_{1} - \phi_{2}))\bigr)
 \end{align}$

In [90]:
%matplotlib inline

In [91]:
# Math functions
import numpy as np

# Solve ODE
from scipy.integrate import odeint, solve_ivp

# Plotting
import matplotlib.pyplot as plt

# Widgets and display
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 
from IPython.display import display
from time import sleep
from matplotlib import animation, rc
from IPython.display import HTML

In [92]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

## Double pendulum class

In [93]:
class doublePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a double pendulum (no driving or damping).
     
    Parameters
    ----------
    L1 : float
        length of the first pendulum
    L2 : float
        length of the second pendulum
    g : float
        gravitational acceleration at the earth's surface
    m1 : float
        mass of first pendulum
    m2 : float
        mass of second pendulum

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., m1=1., m2=1., g=1.
                ):
        self.L1 = L1
        self.L2 = L2
        self.g = g
        self.m1 = m1
        self.m2 = m2
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ---------- 
        y : float
            A 4-component vector with y[0] = phi1(t) and y[1] = phi1_dot
            y[2] = phi2(t) and y[3] = phi2_dot
            
        Returns
        -------
        
        """
        a = (self.m1 + self.m2) * self.L1
        b = self.m2 * self.L2 * np.cos(y[0] - y[2])
        c = -self.m2 * self.L2 * y[3]**2 * np.sin(y[0] - y[2]) - \
            self.g * (self.m1 + self.m2) * np.sin(y[0])  
        d = self.m2 * self.L1 * np.cos(y[0] - y[2])
        e = self.m2 * self.L2
        f = -self.m2 * self.g * np.sin(y[2]) + self.m2 * self.L1 * \
            self.L2 * y[1]**2 * np.sin(y[0] - y[2])
        
        phi1_ddot = (e * c - b * f) / (e * a - d * b)
        phi2_ddot = (a * f - d * c) / (e * a - d * b)
        
        return [y[1], phi1_ddot, y[3], phi2_ddot]
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0,
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0] 
        
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi1, phi1_dot, phi2, phi2_dot = solution.y

        return phi1, phi1_dot, phi2, phi2_dot


## Plotting functions

In [94]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label is passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index
# Labels for individual plot axes
phi_vs_time_labels = (r'$t$', r'$\phi$')
phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt$')
phase_space_labels = (r'$\phi$', r'$d\phi/dt$')

## Set parameters and initial conditions

In [95]:
"""
These values are passed to the widgets so they only need
to be updated in this section
"""
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Pendulum Parameters
L1 = 1.
L2 = 1.
g = 1.
m1 = 1.
m2 = 1.

# initial conditions
phi1_0 = 0.
phi1_dot_0 = 0.

phi2_0 = .05
phi2_dot_0 = 0.

## Generate plots

In [96]:
# This function generates the main output, which is a grid of plots
def pendulum_plots(phi_vs_time_plot=True, phi_dot_vs_time_plot=True, 
                   phase_space_plot=True, L1=L1, L2=L2, m1=m1, m2=m2, g=g,
                   phi1_0 = phi1_0, phi1_dot_0 = phi1_dot_0, phi2_0 = phi2_0, phi2_dot_0 = phi2_dot_0,
                   t_start=t_start, t_end=t_end, delta_t=delta_t, plot_start=0., 
                   phi_vs_time_labels=phi_vs_time_labels, phi_dot_vs_time_labels=phi_dot_vs_time_labels,
                   phase_space_labels=phase_space_labels, animate_flag=False, t_index=0, font_size=18):
    """
    Create plots for interactive_output according to the inputs.
    
    Based on generating a Pendulum instance and the requested graphs.
    """
    
    # add delta_t o it goes at least to t_end (probably should use linspace)
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)  
        
    # Instantiate a pendulum with the passed (or default) parameters
    p1 = doublePendulum(L1=L1, L2=L2, g=g, m1=m1, m2=m2)

    # Solve ODE
    phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)
    
    # Figure out how many rows and columns
    plot_flags = [phi_vs_time_plot, phi_dot_vs_time_plot, phase_space_plot]
    plot_num = plot_flags.count(True)
    plot_rows = 1
    figsize_rows = plot_rows*6
    plot_cols = plot_num
    figsize_cols = min(plot_cols*8, 16)  # at most 16
    
    # Make the plot!
    fig, axes = plt.subplots(plot_rows, plot_cols, 
                             figsize=(figsize_cols,figsize_rows))
    axes = np.atleast_1d(axes)  # make it always a 1d array, even if only 1

    start_index = (np.fabs(t_pts-plot_start)).argmin() # finds nearest index
    
    next_axes = 0
    if phi_vs_time_plot:
        plot_y_vs_x(t_pts, phi1, axis_labels=phi_vs_time_labels, 
                    label=rf'$\phi_{1}$', color = 'blue', title=r'$\phi$ vs. time', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(t_pts, phi2, axis_labels=phi_vs_time_labels, 
                    label=rf'$\phi_{2}$', color = 'red', title=r'$\phi$ vs. time', 
                    ax=axes[next_axes])
        
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], phi1[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(t_pts[t_index], phi2[t_index], 'ro', markersize = '16')
        next_axes += 1
    
    if phi_dot_vs_time_plot:
        plot_y_vs_x(t_pts, phi1_dot, axis_labels=phi_dot_vs_time_labels, 
                    label=rf'$\dot\phi_{1}$', color = 'blue', title=r'$d\phi/dt$ vs. time', 
                    ax=axes[next_axes]) 
        plot_y_vs_x(t_pts, phi2_dot, axis_labels=phi_dot_vs_time_labels, 
                    label=rf'$\dot\phi_{1}$', color = 'red', title=r'$d\phi/dt$ vs. time', 
                    ax=axes[next_axes])
        
        if animate_flag:
            axes[next_axes].plot(t_pts[t_index], phi1_dot[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(t_pts[t_index], phi2_dot[t_index], 'ro', markersize = '16')
        next_axes += 1

    if phase_space_plot:
        plot_y_vs_x(phi1[start_index:-1], phi1_dot[start_index:-1], 
                    axis_labels=phase_space_labels, title='Phase space', color = 'blue', 
                    ax=axes[next_axes])  
        plot_y_vs_x(phi2[start_index:-1], phi2_dot[start_index:-1], 
                    axis_labels=phase_space_labels, title='Phase space', color = 'red', 
                    ax=axes[next_axes])
        
        if animate_flag:
            axes[next_axes].plot(phi1[t_index], phi1_dot[t_index], 'bo', markersize = '16')
            axes[next_axes].plot(phi2[t_index], phi2_dot[t_index], 'ro', markersize = '16')
        next_axes += 1
    
    fig.tight_layout()
    
    return fig, axes


## Create widgets and display

In [97]:
# Widgets for the plot choice (plus a label out front)
plot_choice_w = Label(value='Which plots: ',layout=Layout(width='100px'))
def plot_choice_widget(on=True, plot_description=None):
    """Makes a Checkbox to select whether to show a plot."""
    return Checkbox(value=on, description=plot_description,
                  disabled=False, indent=False, layout=Layout(width='150px'))
phi_vs_time_plot_w = plot_choice_widget(True, r'$\phi$ vs. time')
phi_dot_vs_time_plot_w = plot_choice_widget(False, r'$d\phi/dt$ vs. time')
phase_space_plot_w = plot_choice_widget(True, 'phase space')

# Widgets for the pendulum parameters 
def float_widget(value, min, max, step, description, format):
    """Makes a FloatSlider with the passed parameters and continuous_update
       set to False."""
    slider_border = Layout(border='solid 1.0px')
    return FloatSlider(value=value,min=min,max=max,step=step,disabled=False,
                       description=description,continuous_update=False,
                       orientation='horizontal',layout=slider_border,
                       readout=True,readout_format=format)

L1_w = float_widget(value=L1, min=0.0, max=10., step=0.1,
                        description=r'Length 1:', format='.2f')
L2_w = float_widget(value=L2, min=0.0, max=10., step=0.1,
                        description=r'Length 2:', format='.2f')
g_w = float_widget(value=g, min=0.0, max=10., step=0.05,
                       description=r'Gravity:', format='.2f')
m1_w = float_widget(value=m1,min=0.0,max=10.,step=0.1,
                       description=r'Mass 1:', format='.2f')
m2_w = float_widget(value=m2, min=0, max=10.*np.pi, step=0.1,
                         description=r'Mass 2:', format='.2f')

# Widgets for the initial conditions
phi1_0_w = float_widget(value=phi1_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(\phi_{1})_{0}$:', format='.1f')
phi1_dot_0_w = float_widget(value=phi1_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot\phi_{1})_0$:', format='.1f')
phi2_0_w = float_widget(value=phi2_0, min=0., max=2.*np.pi, step=0.01,
                        description=r'$(\phi_{2})_{0}$:', format='.1f')
phi2_dot_0_w = float_widget(value=phi2_dot_0, min=-10., max=10., step=0.1,
                            description=r'$(\dot\phi_{2})_0$:', format='.1f')

# Widgets for the plotting parameters
t_start_w = float_widget(value=t_start, min=0., max=100., step=10.,
                         description='t start:', format='.1f') 
t_end_w = float_widget(value=t_end, min=.1, max=1000.1, step=.1,
                       description='t end:', format='.1f')
delta_t_w = float_widget(value=delta_t, min=0.01, max=0.1, step=0.01,
                         description='delta t:', format='.2f')
plot_start_w = float_widget(value=0., min=0., max=300., step=5.,
                            description='start plotting:', format='.1f')

# Widgets for the animating
animate_flag_w = Checkbox(value=False, description='Animate',
                  disabled=False, indent=False, layout=Layout(width='100px'))
t_index_w = Play(interval=1, value=0., min=1, max=1000, step=1, 
                      disabled=True, continuous_update=True,
                      description='press play', 
                      orientation='horizontal')

############## Begin: Explicit callback functions #######################

# Make sure that t_end is at least t_start + 50
def update_t_end(*args):
    if t_end_w.value < t_start_w.value:
        t_end_w.value = t_start_w.value + 50     
t_end_w.observe(update_t_end, 'value')
t_start_w.observe(update_t_end, 'value')


# Make sure that plot_start is at least t_start and less than t_end
def update_plot_start(*args):
    if plot_start_w.value < t_start_w.value:
        plot_start_w.value = t_start_w.value
    if plot_start_w.value > t_end_w.value:
        plot_start_w.value = t_end_w.value
plot_start_w.observe(update_plot_start, 'value')
t_start_w.observe(update_plot_start, 'value')
t_end_w.observe(update_plot_start, 'value')

# Only turn on play widget when animate is selected
def turn_on_play_widget(*args):
    if animate_flag_w.value is True:
        t_index_w.disabled = False
        t_pts = np.arange(t_start_w.value, t_end_w.value+delta_t_w.value, 
                          delta_t_w.value)  
        t_index_w.min = (np.fabs(t_pts-plot_start_w.value)).argmin() 
        t_index_w.max = len(t_pts) - 1
    else:
        t_index_w.disabled = True
animate_flag_w.observe(turn_on_play_widget, 'value')


############## End: Explicit callback functions #######################

# Set up the interactive_output widget 
plot_out = widgets.interactive_output(pendulum_plots,
                          dict(
                          phi_vs_time_plot=phi_vs_time_plot_w,
                          phi_dot_vs_time_plot=phi_dot_vs_time_plot_w,
                          phase_space_plot=phase_space_plot_w,
                          L1=L1_w,
                          L2=L2_w,
                          g=g_w,
                          m1=m1_w,
                          m2=m2_w,
                          phi1_0=phi1_0_w,
                          phi2_0=phi2_0_w,
                          phi1_dot_0=phi1_dot_0_w,
                          phi2_dot_0=phi2_dot_0_w,
                          t_start=t_start_w,
                          t_end=t_end_w, 
                          delta_t=delta_t_w,    
                          plot_start=plot_start_w, 
                          animate_flag=animate_flag_w,
                          t_index=t_index_w)
                       )

# Now do some manual layout, where we can put the plot anywhere using plot_out
hbox1 = HBox([plot_choice_w, phi_vs_time_plot_w, phi_dot_vs_time_plot_w,
              phase_space_plot_w]) #  choice of what plots to show
hbox2 = HBox([L1_w, L2_w, g_w, m1_w, m2_w])  # parameters
hbox3 = HBox([phi1_0_w, phi1_dot_0_w, phi2_0_w, phi2_dot_0_w]) # initial conditions
hbox4 = HBox([t_start_w, t_end_w, delta_t_w, plot_start_w]) # time and plot ranges
hbox5 = HBox([animate_flag_w, t_index_w])  # animate

# Set up tabs to organize the controls
tab_height = '70px'  # Fixed minimum height for all tabs.
tab0 = VBox([hbox2, hbox3], layout=Layout(min_height=tab_height))
tab1 = VBox([hbox1, hbox4], layout=Layout(min_height=tab_height))
tab2 = VBox([hbox5], layout=Layout(min_height=tab_height))

tab = Tab(children=[tab0, tab1, tab2])
tab.set_title(0, 'Physics')
tab.set_title(1, 'Plotting')
tab.set_title(2, 'Animate')

# Display widgets
vbox = VBox([tab, plot_out])
display(vbox)

### By adjusting $(\phi_{2})_{0}$ we can demonstrate that the system becomes chaotic for angles larger than the small angle approximation